In [6]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import pandas as pd
import numpy as np

In [8]:
url_base = 'https://www.chicagomag.com/'
url_sub =  'Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base + url_sub
# url

# html = urlopen(url) #403번 에러 발생(부정접근으로 차단-알려진 agent를 헤더에 추가해서 전달하면 됨)
# 헤더를 추가하려면 일반적으로 Request(get_url,headers={key:value})
req = Request(url,headers={'User-Agent':'Mozilla/5.0'})
html=urlopen(req)

soup = BeautifulSoup(html,'html.parser')

soup



<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<title>The 50 Best Sandwiches in Chicago – Chicago Magazine</title>
<style type="text/css">			.heateorSssInstagramBackground{background:radial-gradient(circle at 30% 107%,#fdf497 0,#fdf497 5%,#fd5949 45%,#d6249f 60%,#285aeb 90%)}
						div.heateor_sss_horizontal_sharing i.heateorSssInstagramBackground{background:#000!important;}div.heateor_sss_standard_follow_icons_container i.heateorSssInstagramBackground{background:#000;}
										.heateor_sss_horizontal_sharing .heateorSssSharing,.heateor_sss_standard_follow_icons_container .heateorSssSharing{
							background-color: #000;
							color: #fff;
						border-width: 0px;
			border-style: solid;
			border-color: transparent;
		}
				.heateor_sss_horizontal_sharing .heateorSssTCBackground{
			color:#666;
		}
				.heateor_sss_horizontal_sharing .heateorSssSharing:ho

In [11]:
## 랭킹되어 있는 샌드위치가게 목록 찾아오기
## div class:sammy
soup.find_all('div',{'class':'sammy'}) #리스트로 반환

## 표기방법 2
soup.find_all('div','sammy') #태그이름과 속성값만 전달해도 가능

## 개수 확인
len(soup.find_all('div','sammy'))

50

In [12]:
# 각 원소 접근
soup.find_all('div','sammy')[0]

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
Old Oak Tap<br/>
<em>Read more</em> </a></div>
</div>

In [16]:
## 변수 활용
temp_all = soup.find_all('div','sammy')
type(temp_all)

bs4.element.ResultSet

In [15]:
temp_one = temp_all[0]
type(temp_one) #bs4.element.Tag(bs4 함수를 사용할 수 있음)

bs4.element.Tag

In [18]:
temp_one.find(class_='sammyRank') #class 속성나타내는 함수 인수 class_를 사용해야 함

<div class="sammyRank">1</div>

In [20]:
temp_one.find(class_='sammyRank').get_text()

'1'

In [23]:
temp_one.find(class_='sammyListing').get_text()

'BLT\nOld Oak Tap\nRead more '

In [24]:
temp_one.find('a')['href']

'/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [28]:
# 불용어 정제
tmp_string = temp_one.find(class_='sammyListing').get_text()

In [26]:
# 정규식 패키지정규식 패키지
import re

In [31]:
## 상점 이름 추출 후 정리
re.split(('\n|\r\n'),tmp_string)

['BLT', 'Old Oak Tap', 'Read more ']

In [33]:
from urllib.parse  import urljoin  #문자열을 결합해서 url을 생성, url 형식 체크

In [34]:
urljoin(url_base,temp_one.find('a')['href'])

'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [38]:
# url_base
temp_one.find('a')['href']

'/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [ ]:
'https://www.chicagomag.com/ , /Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/

In [ ]:
###---- 코드 결합 --------------------
## 전체 50개 추출 df 생성
# 각 data를 list에 저장

In [39]:
rank =  []
main_menu = []
cafe_name = []
url_link = []

In [41]:
url_base = 'https://www.chicagomag.com/'
url_sub =  'Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base + url_sub
# url

# html = urlopen(url) #403번 에러 발생(부정접근으로 차단-알려진 agent를 헤더에 추가해서 전달하면 됨)
# 헤더를 추가하려면 일반적으로 Request(get_url,headers={key:value})
req = Request(url,headers={'User-Agent':'Mozilla/5.0'})
html=urlopen(req)

soup = BeautifulSoup(html,'html.parser')
soup_list = soup.find_all('div','sammy')


In [43]:
for item in soup_list :
    rank.append(item.find(class_='sammyRank').get_text())
    tmp_string = item.find(class_='sammyListing').get_text()
    main_menu.append(re.split(('\n|\r\n'),tmp_string)[0])
    cafe_name.append(re.split(('\n|\r\n'),tmp_string)[1])
    url_link.append(urljoin(url_base,item.find('a')['href']))

In [47]:
len(rank),len(main_menu),len(cafe_name),len(url_link)

(50, 50, 50, 50)

In [50]:
## 수집한 자료를 df로 만들어서 csv 저장
data = {'Rank':rank,'Cafe':cafe_name, 'Menu':main_menu, 'URL':url_link}

df= pd.DataFrame(data)
df.head()
df.tail()

,Rank,Cafe,Menu,URL
45,46,Chickpea,Kufta,http://www.chicagomag.com/Chicago-Magazine/Nov...
46,47,The Goddess and Grocer,Debbie’s Egg Salad,http://www.chicagomag.com/Chicago-Magazine/Nov...
47,48,Zenwich,Beef Curry,http://www.chicagomag.com/Chicago-Magazine/Nov...
48,49,Toni Patisserie,Le Végétarien,http://www.chicagomag.com/Chicago-Magazine/Nov...
49,50,Phoebe’s Bakery,The Gatsby,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [52]:
df.to_csv('./수집자료/시카고샌드위치가게.csv',sep=',',encoding='utf-8')

In [ ]:
###--- 데이터 읽어오기

In [53]:
df = pd.read_csv('./수집자료/시카고샌드위치가게.csv',index_col=0)

In [54]:
df.head()

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...


In [56]:
## df의 첫번째 원소의  url 컬럼값을 이용해서 데이터 가져오기
req = Request(df['URL'][0],headers={'User-Agent':'Mozilla/5.0'})
html = urlopen(req)
soup_tmp = BeautifulSoup(html,'html.parser')

temp_string = soup_tmp.find('p','addy').get_text()
temp_string

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [60]:
## 가격 추출(가격 마지막에 . 을 제외)
temp_string.split()[0][:-1]

'$10'

In [61]:
## 주소 데이터 결합
' '.join(temp_string.split()[1:-2])

'2109 W. Chicago Ave.,'

In [62]:
df.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
           dtype='int64')

In [63]:
### 전체 데이터 추출
price = []
address = []

for i in df.index[:3] :
    req = Request(df['URL'][i],headers={'User-Agent':'Mozilla/5.0'})
    html = urlopen(req)
    soup_tmp = BeautifulSoup(html,'html.parser')
    temp_string = soup_tmp.find('p','addy').get_text()
    price.append(temp_string.split()[0][:-1])
    address.append(' '.join(temp_string.split()[1:-2]))

In [65]:
price,address

(['$10', '$9', '$9.50'],
 ['2109 W. Chicago Ave.,', '800 W. Randolph St.,', '445 N. Clark St.,'])

In [ ]:
## 50개  url에 자동접근해서 data 추출
## 상태 진행바 추가

In [66]:
from tqdm import tqdm_notebook

price = []
address = []

for i in tqdm_notebook(df.index) :
    req = Request(df['URL'][i],headers={'User-Agent':'Mozilla/5.0'})
    html = urlopen(req)
    soup_tmp = BeautifulSoup(html,'html.parser')
    temp_string = soup_tmp.find('p','addy').get_text()
    price.append(temp_string.split()[0][:-1])
    address.append(' '.join(temp_string.split()[1:-2]))

<ipython-input-66-4e9bbc80156e>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(df.index) :


In [67]:
## 수집 데이터 df에 추가
df['price']=price
df['address']=address

In [68]:
df.head()

,Rank,Cafe,Menu,URL,price,address
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,"
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,"
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,"
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...,$9.40,"914 Noyes St., Evanston,"
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"825 W. Fulton Mkt.,"


In [69]:
df=df.loc[:,['Rank','Cafe','Menu','price','address']]
df.head()

,Rank,Cafe,Menu,price,address
0,1,Old Oak Tap,BLT,$10,"2109 W. Chicago Ave.,"
1,2,Au Cheval,Fried Bologna,$9,"800 W. Randolph St.,"
2,3,Xoco,Woodland Mushroom,$9.50,"445 N. Clark St.,"
3,4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston,"
4,5,Publican Quality Meats,PB&L,$10,"825 W. Fulton Mkt.,"


In [70]:
## Rank컬럼을 index 로 생성 _ inplace=True -  원본 수정
df.set_index('Rank',inplace=True)

In [71]:
df.tail()

,Cafe,Menu,price,address
Rank,,,,
46,Chickpea,Kufta,$8,"2018 W. Chicago Ave.,"
47,The Goddess and Grocer,Debbie’s Egg Salad,$6.50,"25 E. Delaware Pl.,"
48,Zenwich,Beef Curry,$7.50,"416 N. York St., Elmhurst,"
49,Toni Patisserie,Le Végétarien,$8.75,"65 E. Washington St.,"
50,Phoebe’s Bakery,The Gatsby,$6.85,"3351 N. Broadway,"


In [72]:
## 시카고샌드위치_주소.csv
df.to_csv('./수집자료/시카고샌드위치_주소.csv',sep=',',encoding='utf-8')

In [ ]:
##-- 수집된 주소를 이용해서 각 상점의 위경도 찾기
## 구글맵스틀 이용해서 기본 map  가져오고 위경도 찾기

In [75]:
# 필요패키지 import
import googlemaps #install
import folium
import pandas as pd
import numpy as np

In [77]:
# 구글 클라이언트 등록하는 과정
gmapsKey='구글키'
gmaps = googlemaps.Client(key=gmapsKey)

In [79]:
## 데이터 읽어오기
df = pd.read_csv('./수집자료/시카고샌드위치_주소.csv',index_col=0)
df.head()

,Cafe,Menu,price,address
Rank,,,,
1,Old Oak Tap,BLT,$10,"2109 W. Chicago Ave.,"
2,Au Cheval,Fried Bologna,$9,"800 W. Randolph St.,"
3,Xoco,Woodland Mushroom,$9.50,"445 N. Clark St.,"
4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston,"
5,Publican Quality Meats,PB&L,$10,"825 W. Fulton Mkt.,"


In [81]:
#  첫번째 상점의 위경도 찾기

## 미국 주 이름 앞에는 , 가 와야 함(두번있어도 상관 없음. 단, 없으면 못찾는다)
target_name = df['address'][1] +','+ 'Chicago'
target_name

'2109 W. Chicago Ave.,,Chicago'

In [97]:
# googlemaps 객체의  getcode() 위경도를 찾아옴
g_info = gmaps.geocode(target_name)
g_lo =g_info[0].get("geometry")['location']
g_lo['lat']
# g_lo['lng']

41.8956049

In [100]:
# 50개 위경도 찾아오기
lat=[]
lng=[]

In [101]:
for n in tqdm_notebook(df.index) :
    target_name = df['address'][n] +','+ 'Chicago'
    g_info = gmaps.geocode(target_name)
    g_lo =g_info[0].get("geometry")['location']
    lat.append(g_lo['lat'])
    lng.append(g_lo['lng'])

<ipython-input-101-67e0aeb6ab97>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(df.index) :


In [102]:
len(lat),len(lng)

(50, 50)

In [104]:
lng

[-87.67996149999999,
 -87.6476668,
 -87.6307834,
 -87.6837479,
 -87.64853649999999,
 -87.6253366,
 -87.6252007,
 -87.68287529999999,
 -87.7176727,
 -87.6456492,
 -87.706792,
 -87.64928379999999,
 -87.64464369999999,
 -87.6636812,
 -87.7072408,
 -87.6555453,
 -87.6419289,
 -87.6184837,
 -87.6343113,
 -87.6771273,
 -87.7084498,
 -87.64928379999999,
 -87.66795719999999,
 -87.7027107,
 -87.64928379999999,
 -87.803618,
 -87.64928379999999,
 -87.67557959999999,
 -87.6902192,
 -87.6349487,
 -87.64928379999999,
 -87.6343775,
 -87.64484259999999,
 -87.6778047,
 -87.6592124,
 -87.66934409999999,
 -87.67581620000001,
 -87.6278214,
 -87.64928379999999,
 -87.9396459,
 -87.5938456,
 -87.8414998,
 -87.6343888,
 -87.8022297,
 -87.6264489,
 -87.677857,
 -87.6273926,
 -87.94048839999999,
 -87.6254381,
 -87.6445071]

In [105]:
df['lat']=lat
df['lng']=lng
df.head()

,Cafe,Menu,price,address,lat,lng
Rank,,,,,,
1,Old Oak Tap,BLT,$10,"2109 W. Chicago Ave.,",41.895605,-87.679961
2,Au Cheval,Fried Bologna,$9,"800 W. Randolph St.,",41.884658,-87.647667
3,Xoco,Woodland Mushroom,$9.50,"445 N. Clark St.,",41.890523,-87.630783
4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston,",42.058322,-87.683748
5,Publican Quality Meats,PB&L,$10,"825 W. Fulton Mkt.,",41.886604,-87.648536


In [ ]:
###--- 지도 시각화
## folium map 사용
# map=folium.Map(location='위경도리스트,중심좌표',zoom_start='초기확대정도')

## 마커 표현
# folium.Marker([위도,경도],popup='마커출력글자').add_to(map)

In [107]:
lat_c = df['lat'].mean()
lng_c = df['lng'].mean()

In [109]:
map = folium.Map(location=[lat_c,lng_c],zoom_start=11)
folium.Marker([lat_c,lng_c],popup='Center').add_to(map)
map

In [ ]:
# 전체 cafe의 위치에 Marker 표시하기

In [111]:
map_fin = folium.Map(location=[lat_c,lng_c],zoom_start=11)

In [112]:
for n in df.index :
    folium.Marker([df['lat'][n],df['lng'][n]],
                  popup=df['Cafe'][n]).add_to(map_fin)

In [113]:
map_fin

In [115]:
## 지도 저장
map_fin.save('시카고_카페.html')